**IMPORT PYNGROK TO RUN STREAMLIT ON GOOGLE COLAB**

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


**CREATING APP.PY FILE TO RUN STREAMLIT IN LOCAL SERVER**

In [2]:
%%writefile app.py
import streamlit as st
import numpy as np
from keras.models import load_model
from datetime import date

#  Load the trained LSTM model safely
@st.cache_resource
def load_lstm_model():
    try:
        model = load_model('/content/lstm_sales_model.h5', compile=False)
        return model
    except Exception as e:
        st.error(f" Failed to load model: {e}")
        return None

model = load_lstm_model()

#  Streamlit App UI
st.set_page_config(page_title="LSTM Sales Prediction", layout="centered")
st.title("📈 LSTM Sales Prediction App")
def user_input_features():
    store = st.sidebar.number_input('Store ID', min_value=1, value=1, step=1)
    day_of_week = st.sidebar.selectbox('Day of Week', [1, 2, 3, 4, 5, 6, 7])
    _ = st.sidebar.date_input('Date')  # Not used, kept for UX
    customers = st.sidebar.number_input('Customers', value=500)
    open_status = st.sidebar.radio('Store Open?', [1, 0])
    promo = st.sidebar.radio('Promo Active?', [1, 0])
    state_holiday = st.sidebar.radio('State Holiday?', [0, 1])
    school_holiday = st.sidebar.radio('School Holiday?', [0, 1])

    # Correct shape: (1, 7, 1)
    sequence = np.array([
        [store],
        [day_of_week],
        [customers],
        [open_status],
        [promo],
        [state_holiday],
        [school_holiday]
    ], dtype=np.float32)

    reshaped_input = sequence.reshape(1, 7, 1)

    return reshaped_input

features = user_input_features()

#  Predict Sales
if st.sidebar.button(" Predict Sales"):
    if model is not None:
        try:
            prediction = model.predict(features)
            predicted_sales = float(prediction[0][0])
            st.success(f" Predicted Sales: €{predicted_sales:,.2f}")
        except Exception as e:
            st.error(f" Prediction failed: {e}")
    else:
        st.error(" Model not loaded properly. Check model path and format.")


Writing app.py


**Getting authenticatae token for ngrok and define a funtion to make UI interface in local server **

In [3]:
import threading
import time
from pyngrok import ngrok
ngrok.set_auth_token("2ySL0saJYSgzKIcg1NNACGs2gXP_4SCQYsLJSQ6d5Edt8M3v6")

def run():
    !streamlit run app.py &

# Start Streamlit in a separate thread
thread = threading.Thread(target=run)
thread.start()

# Wait a little for the server to start
time.sleep(3)

# Get public URL
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.28.151:8501

Streamlit app is live at: NgrokTunnel: "https://00f2-35-185-28-151.ngrok-free.app" -> "http://localhost:8501"
